<a href="https://colab.research.google.com/github/karmanandan/ChatPDF/blob/main/ChatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chat with your PDF, with help of OpenAI, FAISS,and LangChain**

In [28]:
# install required libraries

In [ ]:
# https://github.com/john-thuo1/chatWithPDF/blob/main/App.py
# https://medium.com/@johnthuo/chat-with-your-pdf-using-langchain-f-a-i-s-s-and-openai-to-query-pdfs-e7bfde086155

In [ ]:
# !pip install openai
# !pip install PyPDF2
# !pip install langchain
# !pip install tiktoken
# !pip install faiss-cpu

In [29]:
import os
import openai
from PyPDF2 import PdfReader
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [30]:
openai.api_key = ''

In [31]:
def process_text(text):
    # Split the text into chunks using langchain
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    
    # Convert the chunks of text into embeddings to form a knowledge base
    embeddings = OpenAIEmbeddings(openai_api_key =openai.api_key)
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    
    return knowledgeBase

In [32]:
pdf_reader = PdfReader('/content/Austrian AML Act - 2021 (1).pdf')
# Text variable will store the pdf text
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

# Create the knowledge base object
knowledgeBase = process_text(text)

In [33]:
query = input('Ask a question to the PDF')

if query:
    docs = knowledgeBase.similarity_search(query)
    llm = OpenAI(openai_api_key =openai.api_key)
    chain = load_qa_chain(llm, chain_type='stuff')
    
    with get_openai_callback() as cost:
        response = chain.run(input_documents=docs, question=query)
        print('response',response)
        print("\n")
        print('*'*30)
        print("\n")
        print(cost)

Ask a question to the PDFwhat is the purpose of federal act?
response  The purpose of the federal act is to enable the obliged entity or the FMA to fulfil its/their tasks properly for the purposes of this federal act, or to avoid obstructing official or legal inquiries, analyses, investigations or procedures for the purposes of this federal act and to ensure that the prevention, investigation and detection of money laundering and terrorist financing is not jeopardised.


******************************


Tokens Used: 988
	Prompt Tokens: 914
	Completion Tokens: 74
Successful Requests: 1
Total Cost (USD): $0.01976
